In [1]:
from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pwd

/content


In [3]:
cd "drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning/"

/content/drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning


In [4]:
!pwd

/content/drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning


# Imports

In [5]:
import pandas as pd
from collections import Counter
import re
from torch.utils.data import Dataset
import numpy as np
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
!pip3 install pyconll
import pyconll # https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# trainset
train_X = list()
train_y = list()
data = pyconll.load_from_file( './datasets/pt_bosque-ud-train.conllu' )
train_tagged_sentences=[]
t=0
for sentence in data:
    tagged_sentence=[]
    X_sentence = list()
    y_sentence = list()
    for token in sentence:
        if token.upos and token.form:
            t+=1
            tagged_sentence.append((token.form.lower(), token.upos))

            X_sentence.append( token.form )
            y_sentence.append( token.upos )
    train_X.append( ' '.join(str(t) for t in X_sentence) )
    train_y.append( [str(tag) for tag in y_sentence] )
    train_tagged_sentences.append(tagged_sentence)

In [8]:
train_X[0]

'PT em o governo'

In [9]:
train_y[0]

['PROPN', 'ADP', 'DET', 'NOUN']

# CBOW

Baseado em https://colab.research.google.com/drive/1dBIKowtIEhM8MaPgDTkBvPLhikCuxOsU?usp=sharing#scrollTo=IaASmsvNL4vh

In [10]:
treino = train_X

In [11]:
# treino[0].split()

In [12]:
def get_vocab(texts):
  vocab = []
  for row in texts:
    for word in row.split():
      vocab.append( word )
    # vocab.extend(row)

  # vocab = list(set(vocab))
  # vocab += ['<pad>', '<oov>']
  vocab.append( '<pad>' )
  vocab.append( '<oov>' )
  vocab = set(vocab)
  w2id = { w:i for i, w in enumerate(vocab) }
  id2w = { i:w for i, w in enumerate(vocab) }
  return vocab, w2id, id2w

vocab, w2id, id2w = get_vocab(treino)
print('Número de palavras: ', len(vocab))

Número de palavras:  23809


In [13]:
vocab_list = list(vocab)
vocab_list[0]

'alcoolismo'

In [14]:
# vocab_list = list(vocab)
# # vocab_list[ 0 ]

# for i, word in enumerate(vocab_list):
#   if word == '<oov>':
#     print(f'encontrou na posição: {i}')

In [15]:
# vocab_list[12984]

In [16]:
def context_window(sentence, size=3):

  tokens = sentence.split()
  tokens = (['<pad>'] * size) + tokens + (['<pad>'] * size)

  contexts = []
  for i in range(size, len(tokens)-size):
    context = tokens[i-size:i] + tokens[i+1:i+size+1]
    word = tokens[i]
    contexts.append({ 'context': ' '.join(context), 'word': word })
  return contexts

data = []
wsize = 3 # windows size
for sentence in treino:
  data.extend( context_window(sentence, size=wsize) )

data[10]

{'context': 'publicada hoje revela dado supreendente :', 'word': 'um'}

In [17]:
data[110]

{'context': 'com as prioridades o PT .', 'word': 'de'}

In [18]:
import torch
import torch.nn as nn
from torch import optim

class CBOW(nn.Module):
  def __init__(self, embedding_dim, nvocab, window_size, w2id, device):
    '''
    Inicilizando uma rede neural CBOW

    params:
    ---
      embedding_dim: dimensão dos embeddings
      nvocab: tamanho do vocabulário de palavras para as quais treinaremos word embeddings
      window_size: janela de contexto
      w2id: mapping de um token para seu índice na matriz de embeddings
      device: dispositivo onde a rede neural será alocada (e.g. cpu ou cuda)
    '''
    super(CBOW, self).__init__()
    self.device = device
    self.w2id = w2id
    self.lookup = nn.Embedding(nvocab, embedding_dim)
    self.Wb = nn.Linear(2 * window_size * embedding_dim, nvocab)
    self.softmax = nn.LogSoftmax(1)

  def forward(self, X):
    '''
    Forward pass

    params:
    ---
      X: lista de contextos de entrada

    return:
    ---
      probabilidade entre as palavras do vocabulário
    '''
    contexts = []
    for context in X:
      idxs = []
      for token in context.split():
        try:
          idxs.append(w2id[token])
        except:
          idxs.append(w2id['<oov>'])
      contexts.append(idxs)

    contexts = torch.tensor(contexts).to(self.device)
    embeddings = self.lookup(contexts)

    batch_size, window_size, embedding_dim = embeddings.size()
    concatenation = embeddings.view(batch_size, window_size * embedding_dim)

    z = self.Wb(concatenation)
    return self.softmax(z)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_dim = 300
nvocab = len(vocab)
nepochs = 1
batch_size = 256
batch_status = 256
learning_rate = 0.01
window_size = wsize

In [20]:
model = CBOW(embedding_dim, nvocab, window_size=window_size, w2id=w2id, device=device).to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [21]:
traindata = DataLoader(data, batch_size=batch_size, shuffle=True)

In [22]:
for epoch in range(nepochs):
  losses = []
  for batch_idx, row in enumerate(traindata):
    X = row['context']
    y = []
    for word in row['word']:
      try:
        y.append(w2id[word])
      except:
        y.append(w2id['oov'])
    y = torch.tensor(y).to(device)

    # Forward
    outputs = model(X)
    
    # Calculate loss
    loss = criterion(outputs, y)
    losses.append(float(loss))
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Display
    if (batch_idx+1) % batch_status == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(
                            epoch+1, batch_idx + 1, len(traindata),
                            100. * batch_idx / len(traindata), float(loss), 
                            round(sum(losses) / len(losses), 5)))

Train Epoch: 1 [256/672 (38%)]	Loss: 12.224694	Total Loss: 11.654450
Train Epoch: 1 [512/672 (76%)]	Loss: 12.244439	Total Loss: 12.005810


In [23]:
embeddings = model.lookup.weight.data.cpu().numpy()

w2emb = { w:emb for (w, emb) in zip(vocab, list(embeddings))}

Embeddings do <oov> # out of vocabulary

In [24]:
for item in w2emb:
    if item == '<oov>':
      print(w2emb[item])

[ 0.5564119   2.1444917  -0.20005907  0.36524644 -0.76497597 -0.7233162
 -0.7138069   1.6978066  -0.41962638  1.443237   -0.84837484  0.35500878
  0.39613372  0.18711847  1.7530961   1.5089612  -0.27636933 -1.0474018
 -0.22606228  0.06669471  1.052878   -0.8542973   0.7173114  -0.55797875
 -0.6449889  -0.4655621  -0.23992103  0.3673692  -0.27819481 -0.12351319
  1.5860025   0.03737472  0.32384056  1.799033    0.13007745 -0.9245642
  0.56869525  0.81832004 -0.67650473 -0.10205963  1.2199509   1.2841225
 -0.46631333  0.28424415 -0.31432626  0.25387853  1.040279   -0.44382775
 -0.06255431 -0.41091433  2.2101982   0.6796816  -0.33141983  0.02304517
  0.52505606  0.74834347 -2.1222868   0.7208082   0.6900885  -0.10526986
  0.69924265  0.8234417  -0.7287255   0.94828945 -0.0982637  -0.57110894
 -0.42309964  0.84777653 -0.20756602 -1.1941216  -0.68958753  1.3032284
 -1.5807886   1.213773    0.05851575 -1.3904632   0.92844194  0.0347477
  0.46749958 -1.474885    0.8126051  -0.5059858   0.27785

In [25]:
print( len(vocab) )
print( embeddings.shape )

23809
(23809, 300)


In [26]:
vocab = list(vocab)
vocab[0]

'alcoolismo'

In [27]:
embeddings[2][299]

-0.29244775

In [28]:
similarities = cosine_similarity([w2emb['PT']], embeddings)[0]

candidates = sorted(
    [(vocab[i], sim) for i, sim in enumerate(similarities)], 
    key = lambda x: x[1], 
    reverse=True
)[:10]
for cand in candidates:
  print(cand)

('PT', 1.0)
('Adams', 0.2689972)
('corre', 0.22977057)
('Iraque', 0.22664815)
('Económico', 0.21962862)
('seu', 0.21746045)
('legislativo', 0.21477422)
('levanta', 0.21124323)
('eternidade', 0.21048647)
('aroma', 0.20498201)


In [29]:
similarities = cosine_similarity([w2emb['televisão']], embeddings)[0]
  
candidates = sorted(
    [(vocab[i], sim) for i, sim in enumerate(similarities)], 
    key = lambda x: x[1], 
    reverse=True
)[:10]
for cand in candidates:
  print(cand)

('televisão', 1.0)
('porta', 0.24294639)
('Ernest', 0.24153084)
(')', 0.22526795)
('Libreville', 0.22238092)
('bandeiras', 0.21741703)
('elétrica', 0.21710885)
('pena', 0.21193829)
('fomentado', 0.21099864)
('assinantes', 0.21071836)


Daqui pra baixo foi baseado no notebook da professora Aline Paes: 8_3_tagger_pos_v2.ipynb

# Vocab

In [30]:
class Vocab(object):
    def __init__(self, iter, max_size=None, sos_token=None, eos_token=None, unk_token=None):
        """Initialize the vocabulary.
        Args:
            iter: An iterable which produces sequences of tokens used to update
                the vocabulary.
            max_size: (Optional) Maximum number of tokens in the vocabulary.
            sos_token: (Optional) Token denoting the start of a sequence.
            eos_token: (Optional) Token denoting the end of a sequence.
            unk_token: (Optional) Token denoting an unknown element in a
                sequence.
        """
        self.max_size = max_size
        self.pad_token = '<pad>'
        self.sos_token = sos_token
        self.eos_token = eos_token
        self.unk_token = unk_token

        # Add special tokens.
        id2word = [self.pad_token]
        if sos_token is not None:
            id2word.append(self.sos_token)
        if eos_token is not None:
            id2word.append(self.eos_token)
        if unk_token is not None:
            id2word.append(self.unk_token)

        # Update counter with token counts.
        counter = Counter()
        for x in iter:
            counter.update(x)

        # Extract lookup tables.
        if max_size is not None:
            counts = counter.most_common(max_size)
        else:
            counts = counter.items()
            counts = sorted(counts, key=lambda x: x[1], reverse=True)
        words = [x[0] for x in counts]
        id2word.extend(words)
        word2id = {x: i for i, x in enumerate(id2word)}

        self._id2word = id2word
        self._word2id = word2id

    def __len__(self):
        return len(self._id2word)

    def word2id(self, word):
        """Map a word in the vocabulary to its unique integer id.
        Args:
            word: Word to lookup.
        Returns:
            id: The integer id of the word being looked up.
        """
        if word in self._word2id:
            return self._word2id[word]
        elif self.unk_token is not None:
            return self._word2id[self.unk_token]
        else:
            raise KeyError('Word "%s" not in vocabulary.' % word)

    def id2word(self, id):
        """Map an integer id to its corresponding word in the vocabulary.
        Args:
            id: Integer id of the word being looked up.
        Returns:
            word: The corresponding word.
        """
        return self._id2word[id]

# CoNLLDataset e Annotation

In [185]:
class Annotation(object):
    def __init__(self):
        """A helper object for storing annotation data."""
        self.tokens = []
        self.pos_tags = []


class CoNLLDataset(Dataset):
    # def __init__(self, fname, max_exs=None):
    def __init__(self, fname):
        """Initializes the CoNLLDataset.
        Args:
            fname: The .conllu file to load data from.
        """
        self.fname = fname
        # self.annotations = self.process_conll_file(fname, max_exs)
        self.annotations = self.process_conll_file(fname)
        self.token_vocab = Vocab([x.tokens for x in self.annotations],
                                 unk_token='<unk>')
        self.pos_vocab = Vocab([x.pos_tags for x in self.annotations])
        

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        input = [self.token_vocab.word2id(x) for x in annotation.tokens]
        target = [self.pos_vocab.word2id(x) for x in annotation.pos_tags]
        return input, target

    # def process_conll_file(self, fname, max_exs):
    #     # Read the entire file.
    #     with open(fname, 'r') as f:
    #         raw_text = f.read()
    #     # Split into chunks on blank lines.
    #     chunks = re.split(r'^\n', raw_text, flags=re.MULTILINE)
    #     #print(chunks)
    #     # Process each chunk into an annotation.
    #     annotations = []
    #     exs = 0
    #     for chunk in chunks:
    #         if not max_exs or exs < max_exs:
    #             annotation = Annotation()
    #             lines = chunk.split('\n')
    #             # Iterate over all lines in the chunk.
    #             for line in lines:
    #                 # If line is empty ignore it.
    #                 if len(line)==0:
    #                     continue
    #                 # If line is a commend ignore it.
    #                 if line[0] == '#':
    #                     continue
    #                 # Otherwise split on tabs and retrieve the token and the
    #                 # POS tag fields.
    #                 fields = line.split('\t')
    #                 annotation.tokens.append(fields[1])
    #                 annotation.pos_tags.append(fields[3])
    #             if (len(annotation.tokens) > 0) and (len(annotation.pos_tags) > 0):
    #                 annotations.append(annotation)
    #         exs += 1
    #     return annotations

    def process_conll_file(self, fname):
        # print(f'fname: {fname}')
        data = pyconll.load_from_file( fname )
        annotations = []

        for sentence in data:
            annotation = Annotation()
            tokens_list = []
            pos_list = []
            
            for token in sentence:
                if token.upos and token.form:
                    # print(f'token: {type(token)}')
                    # print(f'token.form: {token.form}')
                    # print(f'token.upos: {token.upos}')
                    tokens_list.append( token.form )
                    pos_list.append( token.upos )
            
            annotation.tokens.append( tokens_list )
            annotation.pos_tags.append( pos_list )
                        
            annotation.tokens = annotation.tokens[0]
            annotation.pos_tags = annotation.pos_tags[0]
            # print(f'annotation.tokens: {annotation.tokens}')
            # print(f'annotation.pos_tags: {annotation.pos_tags}')
            # print('-----------------------------------------')

            annotations.append(annotation)

        return annotations

In [186]:
dataset_temp = CoNLLDataset('./datasets/pt_bosque-ud-train.conllu')

In [187]:
dataset_temp.annotations[1].tokens

['BRASÍLIA',
 'Pesquisa',
 'Datafolha',
 'publicada',
 'hoje',
 'revela',
 'um',
 'dado',
 'supreendente',
 ':',
 'recusando',
 'uma',
 'postura',
 'radical',
 ',',
 'a',
 'esmagadora',
 'maioria',
 '(',
 '77',
 '%',
 ')',
 'de',
 'os',
 'eleitores',
 'quer',
 'o',
 'PT',
 'participando',
 'de',
 'o',
 'Governo',
 'Fernando',
 'Henrique',
 'Cardoso',
 '.']

In [188]:
dataset_temp.annotations[1].pos_tags

['PROPN',
 'PROPN',
 'PROPN',
 'VERB',
 'ADV',
 'VERB',
 'DET',
 'NOUN',
 'ADJ',
 'PUNCT',
 'VERB',
 'DET',
 'NOUN',
 'ADJ',
 'PUNCT',
 'DET',
 'ADJ',
 'NOUN',
 'PUNCT',
 'NUM',
 'SYM',
 'PUNCT',
 'ADP',
 'DET',
 'NOUN',
 'VERB',
 'DET',
 'PROPN',
 'VERB',
 'ADP',
 'DET',
 'NOUN',
 'PROPN',
 'PROPN',
 'PROPN',
 'PUNCT']

In [189]:
len(dataset_temp.annotations)

7018

# Funções: pad() e collate_annotations()

In [191]:
def pad(sequences, max_length, pad_value=0):
    """Pads a list of sequences.
    Args:
        sequences: A list of sequences to be padded.
        max_length: The length to pad to.
        pad_value: The value used for padding.
    Returns:
        A list of padded sequences.
    """
    out = []
    for sequence in sequences:
        padded = sequence + [0]*(max_length - len(sequence))
        out.append(padded)
    return out


def collate_annotations(batch):
    """Function used to collate data returned by CoNLLDataset."""
    # Get inputs, targets, and lengths.
    inputs, targets = zip(*batch)
    lengths = [len(x) for x in inputs]
    # Sort by length.
    sort = sorted(zip(inputs, targets, lengths),
                  key=lambda x: x[2],
                  reverse=True)
    inputs, targets, lengths = zip(*sort)
    # Pad.
    max_length = max(lengths)
    inputs = pad(inputs, max_length)
    targets = pad(targets, max_length)
    # Transpose.
    inputs = list(map(list, zip(*inputs)))
    targets = list(map(list, zip(*targets)))
    # Convert to PyTorch variables.
    inputs = Variable(torch.LongTensor(inputs))
    targets = Variable(torch.LongTensor(targets))
    lengths = Variable(torch.LongTensor(lengths))
    if torch.cuda.is_available():
        inputs = inputs.cuda()
        targets = targets.cuda()
        lengths = lengths.cuda()
    return inputs, targets, lengths

# Tagger - GRU

In [192]:

from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Tagger(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim,
                 n_layers, 
                 embedding_dim=64,
                 hidden_dim=64,
                 dropout=0.5,
                 bidirectional=True,
                 pad_idx=0):
        """Initializes the tagger.
        
        Args:
            input_dim: Size of the input vocabulary, projection
            output_dim: Size of the output vocabulary.
            embedding_dim: Dimension of the word embeddings.
            hidden_dim: Number of units in each LSTM hidden layer.
            bidirectional: Whether or not to use a bidirectional rnn.
        """
        super(Tagger, self).__init__()

        # Store parameters
        self.input_dim = input_dim 
        self.output_dim = output_dim
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
          
        # Define layers
        self.word_embeddings = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers = n_layers, 
                          bidirectional=bidirectional,
                          dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.activation = nn.LogSoftmax(dim=2)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths=None, hidden=None):
        """Computes a forward pass of the language model.
        
        Args:
            x: A LongTensor w/ dimension [seq_len, batch_size].
            lengths: The lengths of the sequences in x.
            hidden: Hidden state to be fed into the lstm.
            
        Returns:
            net: Probability of the next word in the sequence.
            hidden: Hidden state of the lstm.
        """
        seq_len, batch_size = x.size()
        
        # If no hidden state is provided, then default to zeros.
        if hidden is None:
            if self.bidirectional:
                num_directions = 2
            else:
                num_directions = 1
            hidden = Variable(torch.zeros(num_directions, batch_size, self.hidden_dim))
            if torch.cuda.is_available():
                hidden = hidden.cuda()

        net = self.word_embeddings(x)
        # Pack before feeding into the RNN.
        if lengths is not None:
            lengths = lengths.data.view(-1).tolist()
            net = pack_padded_sequence(net, lengths)
        net, hidden = self.rnn(net, hidden)
        # Unpack after
        if lengths is not None:
            net, _ = pad_packed_sequence(net)
        net = self.fc(net)
        net = self.activation(net)

        return net, hidden

# Training Model

In [193]:
# Load datasets.
# train_dataset = CoNLLDataset('./datasets/pt_bosque-ud-train.conllu', 4096)
# dev_dataset = CoNLLDataset('./datasets/pt_bosque-ud-dev.conllu', 1024)
train_dataset = CoNLLDataset('./datasets/pt_bosque-ud-train.conllu')
dev_dataset = CoNLLDataset('./datasets/pt_bosque-ud-dev.conllu')

Substituindo os ids das palavras pela representação de embeddings.

In [218]:
train_dataset_embeddings = []

for sentence in train_dataset:

    # print(f'sentence: {sentence}')
    setence_words_embeddings = []
    setence_pos_tags = []

    for i in range(0, len(sentence[0])):

        word_id = sentence[0][i]
        pos_tag = sentence[1][i]

        # print(f'i: {i}')
        # print(f'word_id: {word_id}')
        # print(f'pos_tag: {pos_tag}')

        # print(f'{word_id} - {train_dataset.token_vocab.id2word( word_id )}')
        word = train_dataset.token_vocab.id2word( word_id )
        if word in w2emb:
            embeddings = w2emb[ word ]
        else:
            embeddings = w2emb['<oov>']

        embeddings = list(embeddings)
        # print(f'embeddings: {embeddings}')
        # print(f'{word_id} - {word} - {embeddings}')

        setence_words_embeddings.append(embeddings)
        setence_pos_tags.append(pos_tag)
        # print(f'setence_words_embeddings: {setence_words_embeddings}')
        # print(f'setence_pos_tags: {setence_pos_tags}')
        # print(f'len(setence_words_embeddings): {len(setence_words_embeddings)}')
        # print(f'len(setence_pos_tags): {len(setence_pos_tags)}')

        new_sentence_embdeddings_pos = ((setence_words_embeddings, setence_pos_tags))
        # print(f'sentence: {sentence}')
        # print(f'new_sentence_embdeddings_pos: {new_sentence_embdeddings_pos}')
        train_dataset_embeddings.append( new_sentence_embdeddings_pos )

        # print('------------------------------------------------')

In [219]:
train_dataset[0][0]

[396, 7, 5, 107]

In [220]:
train_dataset_embeddings[0][0]

[[0.06372718,
  1.081287,
  -0.2566062,
  0.35581657,
  0.20967016,
  -0.36711636,
  0.366946,
  -0.3230368,
  -0.34950674,
  0.4455702,
  -0.7013678,
  0.20763332,
  0.7158997,
  0.07245133,
  0.05840904,
  1.7177013,
  1.2872624,
  0.007037018,
  -0.36843827,
  0.30398393,
  1.9073563,
  0.046871826,
  0.362007,
  0.19549051,
  -0.055350173,
  -0.018726848,
  -0.70453614,
  0.5320796,
  0.21127453,
  1.4585506,
  0.15391096,
  -0.99725866,
  0.109845065,
  -0.020156017,
  0.96515375,
  1.9662554,
  0.8686647,
  0.0358276,
  0.13300346,
  0.0016626305,
  0.2117124,
  -0.42018014,
  0.069936104,
  -0.3920737,
  0.6753673,
  0.28072768,
  0.38520417,
  -0.23346636,
  -0.09435507,
  -0.08085937,
  -0.007921909,
  -0.049984373,
  1.0996103,
  -0.12756191,
  -0.57305413,
  -0.0906943,
  -0.18224198,
  -0.71684444,
  0.30599272,
  -0.9337199,
  -0.3402406,
  0.08233886,
  -0.12936674,
  -0.048676677,
  -1.148141,
  -0.265937,
  0.14547049,
  0.20106123,
  0.7871115,
  0.0467316,
  0.3700147

In [221]:
train_dataset[0][1]

[6, 3, 2, 1]

In [222]:
train_dataset_embeddings[0][1]

[6, 3, 2, 1]

In [224]:
dev_dataset_embeddings = []

for sentence in dev_dataset:

    # print(f'sentence: {sentence}')
    setence_words_embeddings = []
    setence_pos_tags = []

    for i in range(0, len(sentence[0])):

        word_id = sentence[0][i]
        pos_tag = sentence[1][i]

        # print(f'i: {i}')
        # print(f'word_id: {word_id}')
        # print(f'pos_tag: {pos_tag}')

        # print(f'{word_id} - {dev_dataset.token_vocab.id2word( word_id )}')
        word = dev_dataset.token_vocab.id2word( word_id )
        if word in w2emb:
            embeddings = w2emb[ word ]
        else:
            embeddings = w2emb['<oov>']

        embeddings = list(embeddings)
        # print(f'embeddings: {embeddings}')
        # print(f'{word_id} - {word} - {embeddings}')

        setence_words_embeddings.append(embeddings)
        setence_pos_tags.append(pos_tag)
        # print(f'setence_words_embeddings: {setence_words_embeddings}')
        # print(f'setence_pos_tags: {setence_pos_tags}')
        # print(f'len(setence_words_embeddings): {len(setence_words_embeddings)}')
        # print(f'len(setence_pos_tags): {len(setence_pos_tags)}')

        new_sentence_embdeddings_pos = ((setence_words_embeddings, setence_pos_tags))
        # print(f'sentence: {sentence}')
        # print(f'new_sentence_embdeddings_pos: {new_sentence_embdeddings_pos}')
        dev_dataset_embeddings.append( new_sentence_embdeddings_pos )

        # print('------------------------------------------------')

In [227]:
train_dataset_embeddings[0] # primeira sentença: embeddings das palavras + pos_tag

([[0.06372718,
   1.081287,
   -0.2566062,
   0.35581657,
   0.20967016,
   -0.36711636,
   0.366946,
   -0.3230368,
   -0.34950674,
   0.4455702,
   -0.7013678,
   0.20763332,
   0.7158997,
   0.07245133,
   0.05840904,
   1.7177013,
   1.2872624,
   0.007037018,
   -0.36843827,
   0.30398393,
   1.9073563,
   0.046871826,
   0.362007,
   0.19549051,
   -0.055350173,
   -0.018726848,
   -0.70453614,
   0.5320796,
   0.21127453,
   1.4585506,
   0.15391096,
   -0.99725866,
   0.109845065,
   -0.020156017,
   0.96515375,
   1.9662554,
   0.8686647,
   0.0358276,
   0.13300346,
   0.0016626305,
   0.2117124,
   -0.42018014,
   0.069936104,
   -0.3920737,
   0.6753673,
   0.28072768,
   0.38520417,
   -0.23346636,
   -0.09435507,
   -0.08085937,
   -0.007921909,
   -0.049984373,
   1.0996103,
   -0.12756191,
   -0.57305413,
   -0.0906943,
   -0.18224198,
   -0.71684444,
   0.30599272,
   -0.9337199,
   -0.3402406,
   0.08233886,
   -0.12936674,
   -0.048676677,
   -1.148141,
   -0.265937,

In [228]:
train_dataset_embeddings[0][0] # primeira sentença: embeddings das palavras

[[0.06372718,
  1.081287,
  -0.2566062,
  0.35581657,
  0.20967016,
  -0.36711636,
  0.366946,
  -0.3230368,
  -0.34950674,
  0.4455702,
  -0.7013678,
  0.20763332,
  0.7158997,
  0.07245133,
  0.05840904,
  1.7177013,
  1.2872624,
  0.007037018,
  -0.36843827,
  0.30398393,
  1.9073563,
  0.046871826,
  0.362007,
  0.19549051,
  -0.055350173,
  -0.018726848,
  -0.70453614,
  0.5320796,
  0.21127453,
  1.4585506,
  0.15391096,
  -0.99725866,
  0.109845065,
  -0.020156017,
  0.96515375,
  1.9662554,
  0.8686647,
  0.0358276,
  0.13300346,
  0.0016626305,
  0.2117124,
  -0.42018014,
  0.069936104,
  -0.3920737,
  0.6753673,
  0.28072768,
  0.38520417,
  -0.23346636,
  -0.09435507,
  -0.08085937,
  -0.007921909,
  -0.049984373,
  1.0996103,
  -0.12756191,
  -0.57305413,
  -0.0906943,
  -0.18224198,
  -0.71684444,
  0.30599272,
  -0.9337199,
  -0.3402406,
  0.08233886,
  -0.12936674,
  -0.048676677,
  -1.148141,
  -0.265937,
  0.14547049,
  0.20106123,
  0.7871115,
  0.0467316,
  0.3700147

In [229]:
train_dataset_embeddings[0][1] # primeira sentença: pos_tag

[6, 3, 2, 1]

In [230]:
data_loader = DataLoader(train_dataset_embeddings, batch_size=batch_size, shuffle=True,
                         collate_fn=collate_annotations)

In [231]:
data_loader

In [232]:
count = 0
for inputs, targets, lengths in data_loader:
    if count >= 10: 
        break;
    else:
        print( f'inputs.shape: {inputs.shape}' )
        print( f'inputs: {inputs}' )
        print( f'targets: {targets}' )
        print( f'lengths: {lengths}' )

    count = count + 1
    print('------------------------------------------')

TypeError: ignored

In [48]:
dev_dataset.token_vocab = train_dataset.token_vocab
dev_dataset.pos_vocab = train_dataset.pos_vocab

# Hyperparameters / constants.
input_vocab_size = len(train_dataset.token_vocab)
output_vocab_size = len(train_dataset.pos_vocab)
batch_size = 16
epochs = 1
n_layers = 1

# Initialize the model.
# model = Tagger(input_vocab_size, output_vocab_size, n_layers)
model = Tagger(
    input_dim = input_vocab_size,
    output_dim = output_vocab_size, 
    n_layers = n_layers,
    embedding_dim = embedding_dim) # daniel
if torch.cuda.is_available():
    model = model.cuda()

# Loss function weights.
weight = torch.ones(output_vocab_size)
weight[0] = 0
if torch.cuda.is_available():
    weight = weight.cuda()
    
# Initialize loss function and optimizer.
loss_function = torch.nn.NLLLoss(weight)
optimizer = torch.optim.Adam(model.parameters())

# Main training loop.
# data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
#                          collate_fn=collate_annotations)
# dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
#                         collate_fn=collate_annotations)
data_loader = DataLoader(train_dataset_embeddings, batch_size=batch_size, shuffle=True,
                         collate_fn=collate_annotations)
dev_loader = DataLoader(dev_dataset_embeddings, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_annotations)

losses = []
i = 0
for epoch in range(epochs):
    for inputs, targets, lengths in data_loader:
        optimizer.zero_grad()
        # outputs, _ = model(inputs, lengths=lengths)
        outputs, _ = model(inputs[0], lengths=lengths) # daniel

        outputs = outputs.view(-1, output_vocab_size)
        # print('---------------outputs----------------')
        # print(outputs.shape)
        # print(outputs)
        targets = targets.view(-1)
        # print('---------------targets----------------')
        # print(targets.shape)
        # print(targets)

        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        #losses.append(loss.data[0])
        losses.append(loss.item())
        if (i % 10) == 0:
            # Compute dev loss over entire dev set.
            # NOTE: This is expensive. You may want to only use a 
            # subset of the dev set.
            #print('iteration, ', i)
            dev_losses = []
            for inputs, targets, lengths in dev_loader:
                outputs, _ = model(inputs, lengths=lengths)
                outputs = outputs.view(-1, output_vocab_size)
                targets = targets.view(-1)
                loss = loss_function(outputs, targets)
                dev_losses.append(loss.item())
            avg_train_loss = np.mean(losses)
            avg_dev_loss = np.mean(dev_losses)
            losses = []
            #print('here')
            print('Epoch %i Iteration %i - Train Loss: %0.6f - Dev Loss: %0.6f' % (epoch, i, avg_train_loss, avg_dev_loss), end='\n')
            torch.save(model, 'pos_tagger_gru.pt')
        i += 1
        
torch.save(model, 'pos_tagger_gru.final.pt')

RuntimeError: ignored

In [ ]:
train_dataset[0]

In [ ]:
# Collect the predictions and targets
y_true = []
y_pred = []

for inputs, targets, lengths in dev_loader:
    outputs, _ = model(inputs, lengths=lengths)
    _, preds = torch.max(outputs, dim=2)
    targets = targets.view(-1)
    preds = preds.view(-1)
    if torch.cuda.is_available():
        targets = targets.cpu()
        preds = preds.cpu()
    y_true.append(targets.data.numpy())
    y_pred.append(preds.data.numpy())

In [ ]:
# Stack into numpy arrays
y_real = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

In [ ]:
y_real_temp = []
for id in y_real:
  y_real_temp.append(dev_dataset.pos_vocab._id2word[ id ])
y_real = y_real_temp
print(y_real)

In [ ]:
y_pred_temp = []
for id in y_pred:
  y_pred_temp.append(dev_dataset.pos_vocab._id2word[ id ])
y_pred = y_pred_temp
print(y_pred)

In [ ]:
len(y_real)

In [ ]:
len(y_pred)

In [ ]:
print( classification_report( y_real, y_pred ) )
f1 = f1_score( y_real, y_pred, average='weighted' )
acc = accuracy_score( y_real, y_pred )
print( f'F1: {f1:.2}' )
print( f'Accuracy: {acc:.2}' )

# # Compute accuracy
# acc = np.mean(y_real[y_real != 0] == y_pred[y_real != 0])
# print('Accuracy - %0.6f\n' % acc)

# # Evaluate f1-score
# from sklearn.metrics import f1_score
# score = f1_score(y_real, y_pred, average=None)
# print('F1-scores:\n')
# for label, score in zip(dev_dataset.pos_vocab._id2word[1:], score[1:]):
#     print('%s - %0.6f' % (label, score))

In [ ]:
model = torch.load('pos_tagger_gru.final.pt')

def inference(sentence):
    # Convert words to id tensor.
    ids = [[dataset.token_vocab.word2id(x)] for x in sentence]
    ids = Variable(torch.LongTensor(ids))
    if torch.cuda.is_available():
        ids = ids.cuda()
    # Get model output.
    output, _ = model(ids)
    _, preds = torch.max(output, dim=2)
    if torch.cuda.is_available():
        preds = preds.cpu()
    preds = preds.data.view(-1).numpy()
    pos_tags = [dataset.pos_vocab.id2word(x) for x in preds]
    for word, tag in zip(sentence, pos_tags):
        print('%s - %s' % (word, tag))

In [ ]:
def inference_with_labels(sentence, labels):
    #print(sentence)
    # Convert words to id tensor.
    ids = [[dataset.token_vocab.word2id(x)] for x in sentence]
    print(ids)
    ids = Variable(torch.LongTensor(ids))
    if torch.cuda.is_available():
        ids = ids.cuda()
    # Get model output.
    output, _ = model(ids)
    _, preds = torch.max(output, dim=2)
    if torch.cuda.is_available():
        preds = preds.cpu()
    preds = preds.data.view(-1).numpy()
    pos_tags = [dataset.pos_vocab.id2word(x) for x in preds]
    #labels = [dataset.pos_vocab.id2word(x) for x in labels]
    #sentence = [test_dataset.token_vocab.id2word(x) for x in ids]
    for word, tag, label in zip(sentence, pos_tags, labels):
        print('%s - %s - %s' % (word, tag, label))

In [ ]:
test_dataset = CoNLLDataset('./datasets/pt_bosque-ud-test.conllu')
dataset = CoNLLDataset('./datasets/pt_bosque-ud-train.conllu')

sentence, labels = test_dataset[10]
sentence = [test_dataset.token_vocab.id2word(x) for x in sentence]
print(sentence)
labels = [test_dataset.pos_vocab.id2word(x) for x in labels]
inference_with_labels(sentence, labels)